# 40. 係り受け解析結果の読み込み（形態素）

形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，係り受け解析の結果（ai.ja.txt.parsed）を読み込み，各文をMorphオブジェクトのリストとして表現し，冒頭の説明文の形態素列を表示せよ．

In [1]:
class Morph:
    def __init__(self, morph):
        
        fields = line.split('\t')
        attrs = fields[1].split(',')
        
        self.surface = fields[0]
        self.base = attrs[6]
        self.pos = attrs[0]
        self.pos1 = attrs[1]

sentences = []
morphs = []

with open('ai.ja.txt.parsed', encoding='utf-8') as f:
    for line in f:
        if '\t' in line:
            morphs.append(Morph(line))
        elif line == 'EOS\n':
            if len(morphs) > 0:
                sentences.append(morphs)
                morphs = []

[vars(morph) for morph in sentences[1]]

[{'surface': '人工', 'base': '人工', 'pos': '名詞', 'pos1': '一般'},
 {'surface': '知能', 'base': '知能', 'pos': '名詞', 'pos1': '一般'},
 {'surface': '（', 'base': '（', 'pos': '記号', 'pos1': '括弧開'},
 {'surface': 'じん', 'base': 'じん', 'pos': '名詞', 'pos1': '一般'},
 {'surface': 'こうち', 'base': 'こうち', 'pos': '名詞', 'pos1': '一般'},
 {'surface': 'のう', 'base': 'のう', 'pos': '助詞', 'pos1': '終助詞'},
 {'surface': '、', 'base': '、', 'pos': '記号', 'pos1': '読点'},
 {'surface': '、', 'base': '、', 'pos': '記号', 'pos1': '読点'},
 {'surface': 'AI', 'base': '*\n', 'pos': '名詞', 'pos1': '一般'},
 {'surface': '〈', 'base': '〈', 'pos': '記号', 'pos1': '括弧開'},
 {'surface': 'エーアイ', 'base': '*\n', 'pos': '名詞', 'pos1': '固有名詞'},
 {'surface': '〉', 'base': '〉', 'pos': '記号', 'pos1': '括弧閉'},
 {'surface': '）', 'base': '）', 'pos': '記号', 'pos1': '括弧閉'},
 {'surface': 'と', 'base': 'と', 'pos': '助詞', 'pos1': '格助詞'},
 {'surface': 'は', 'base': 'は', 'pos': '助詞', 'pos1': '係助詞'},
 {'surface': '、', 'base': '、', 'pos': '記号', 'pos1': '読点'},
 {'surface': '「', 'base': '

# 41. 係り受け解析結果の読み込み（文節・係り受け）

40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストの係り受け解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，冒頭の説明文の文節の文字列と係り先を表示せよ．本章の残りの問題では，ここで作ったプログラムを活用せよ．

In [47]:
class Chunk:
    def __init__(self, morphs, dst, srcs):        
        self.morphs = morphs
        self.dst = dst
        self.srcs = srcs
    def __str__(self):
        return ''.join([morph.surface for morph in self.morphs if morph.pos != '記号'])
    def contain_pos(self, pos):
        return pos in [morph.pos for morph in self.morphs]
    def is_noun_phrase(self):
        return '名詞' in [morph.pos for morph in self.morphs] and self.morphs[-1].pos == '助詞'

sentences = []
chunks = []
morphs = []        
dst = -2

with open('ai.ja.txt.parsed', encoding='utf-8') as f:
    for line in f:        
        if line == 'EOS\n':
            if dst != -2:
                chunks.append(Chunk(morphs, dst, []))
            if len(chunks) > 0:
                #各文節のsrcsを生成
                for i in range(len(chunks)):
                    if chunks[i].dst != -1:
                        chunks[chunks[i].dst].srcs.append(i)
                sentences.append(chunks)
            #初期化
            chunks = []
            morphs = []
            dst = -2
        elif '\t' in line: #形態素
            morphs.append(Morph(line))
        else: #係り受け情報
            if dst != -2:
                chunks.append(Chunk(morphs, dst, []))
            morphs = []
            dst = int(line.split(' ')[2].strip('D')) 

for chunk in sentences[1]:
    print([morph.surface for morph in chunk.morphs], chunk.dst, chunk.srcs)

['人工', '知能'] 17 []
['（', 'じん', 'こうち', 'のう', '、', '、'] 17 []
['AI'] 3 []
['〈', 'エーアイ', '〉', '）', 'と', 'は', '、'] 17 [2]
['「', '『', '計算'] 5 []
['（', '）', '』', 'という'] 9 [4]
['概念', 'と'] 9 []
['『', 'コンピュータ'] 8 []
['（', '）', '』', 'という'] 9 [7]
['道具', 'を'] 10 [5, 6, 8]
['用い', 'て'] 12 [9]
['『', '知能', '』', 'を'] 12 []
['研究', 'する'] 13 [10, 11]
['計算', '機', '科学'] 14 [12]
['（', '）', 'の'] 15 [13]
['一', '分野', '」', 'を'] 16 [14]
['指す'] 17 [15]
['語', '。'] 34 [0, 1, 3, 16]
['「', '言語', 'の'] 20 []
['理解', 'や'] 20 []
['推論', '、'] 21 [18, 19]
['問題', '解決', 'など', 'の'] 22 [20]
['知的', '行動', 'を'] 24 [21]
['人間', 'に'] 24 []
['代わっ', 'て'] 26 [22, 23]
['コンピューター', 'に'] 26 []
['行わ', 'せる'] 27 [24, 25]
['技術', '」', '、', 'または', '、'] 34 [26]
['「', '計算', '機'] 29 []
['（', 'コンピュータ', '）', 'による'] 31 [28]
['知的', 'な'] 31 []
['情報処理', 'システム', 'の'] 33 [29, 30]
['設計', 'や'] 33 []
['実現', 'に関する'] 34 [31, 32]
['研究', '分野', '」', 'と', 'も'] 35 [17, 27, 33]
['さ', 'れる', '。'] -1 [34]


# 42. 係り元と係り先の文節の表示

係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [3]:
chuncks = sentences[1]

for chunck in chuncks:
    if chunck.dst != -1:
        print(chunck, '\t', chuncks[chunck.dst])

人工知能 	 語
じんこうちのう 	 語
AI 	 エーアイとは
エーアイとは 	 語
計算 	 という
という 	 道具を
概念と 	 道具を
コンピュータ 	 という
という 	 道具を
道具を 	 用いて
用いて 	 研究する
知能を 	 研究する
研究する 	 計算機科学
計算機科学 	 の
の 	 一分野を
一分野を 	 指す
指す 	 語
語 	 研究分野とも
言語の 	 推論
理解や 	 推論
推論 	 問題解決などの
問題解決などの 	 知的行動を
知的行動を 	 代わって
人間に 	 代わって
代わって 	 行わせる
コンピューターに 	 行わせる
行わせる 	 技術または
技術または 	 研究分野とも
計算機 	 コンピュータによる
コンピュータによる 	 情報処理システムの
知的な 	 情報処理システムの
情報処理システムの 	 実現に関する
設計や 	 実現に関する
実現に関する 	 研究分野とも
研究分野とも 	 される


# 43. 名詞を含む文節が動詞を含む文節に係るものを抽出

名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [4]:
for chunck in chuncks:
    target = chuncks[chunck.dst]
    if chunck.contain_pos('名詞') and target.contain_pos('動詞'):
        print(chunck, '\t', target)

道具を 	 用いて
知能を 	 研究する
一分野を 	 指す
知的行動を 	 代わって
人間に 	 代わって
コンピューターに 	 行わせる
研究分野とも 	 される


# 44. 係り受け木の可視化

与えられた文の係り受け木を有向グラフとして可視化せよ．可視化には，Graphviz等を用いるとよい．

In [6]:
from graphviz import Digraph

dot = Digraph(name="tree", format="png")

for i, chunck in enumerate(chuncks):
    dot.node(name=str(i), label=str(chunck), fontname="Microsoft YaHei")

for i, chunck in enumerate(chuncks):
    if chunck.dst != -1:
        dot.edge(str(i), str(chunck.dst))

#print(dot.source)
dot.view(filename="tree", directory="D:\\")

'D:\\tree.png'

# 45. 動詞の格パターンの抽出

今回用いている文章をコーパスと見なし，日本語の述語が取りうる格を調査したい． 動詞を述語，動詞に係っている文節の助詞を格と考え，述語と格をタブ区切り形式で出力せよ． ただし，出力は以下の仕様を満たすようにせよ．

    動詞を含む文節において，最左の動詞の基本形を述語とする
    述語に係る助詞を格とする
    述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる

「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． この文は「作り出す」という１つの動詞を含み，「作り出す」に係る文節は「ジョン・マッカーシーは」，「会議で」，「用語を」であると解析された場合は，次のような出力になるはずである．

作り出す	で は を

このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．

    コーパス中で頻出する述語と格パターンの組み合わせ
    「行う」「なる」「与える」という動詞の格パターン（コーパス中で出現頻度の高い順に並べよ）


In [35]:
ans = []

for chuncks in sentences:
    for chunck in chuncks:
        for morph in chunck.morphs:
            if morph.pos == '動詞':
                verb = morph.base
                particles = []
                for src in chunck.srcs:
                    
                    #すべての助詞を記入するなら
                    # for morph in chuncks[src].morphs:
                    #     if morph.pos == '助詞':
                    #         particles.append(morph.surface)
                    
                    #文節末の助詞だけを記入するなら
                    morph = chuncks[src].morphs[-1]
                    if morph.pos == '助詞':
                        particles.append(morph.surface)
                
                if len(particles) > 0:
                    particles = sorted(particles)
                    ans.append(verb + '\t' + ' '.join(particles))
                
                break #最左の動詞だけを取る
                
with open('.\\45.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(ans))

# 46. 動詞の格フレーム情報の抽出

45のプログラムを改変し，述語と格パターンに続けて項（述語に係っている文節そのもの）をタブ区切り形式で出力せよ．45の仕様に加えて，以下の仕様を満たすようにせよ．

    項は述語に係っている文節の単語列とする（末尾の助詞を取り除く必要はない）
    述語に係る文節が複数あるときは，助詞と同一の基準・順序でスペース区切りで並べる

「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． この文は「作り出す」という１つの動詞を含み，「作り出す」に係る文節は「ジョン・マッカーシーは」，「会議で」，「用語を」であると解析された場合は，次のような出力になるはずである．

作り出す	で は を	会議で ジョンマッカーシーは 用語を


In [36]:
ans = []

for chuncks in sentences:
    for chunck in chuncks:
        for morph in chunck.morphs:
            if morph.pos == '動詞':
                verb = morph.base
                arguments = []
                for src in chunck.srcs:
                    
                    #すべての助詞を記入するなら
                    # for morph in chuncks[src].morphs:
                    #     if morph.pos == '助詞':
                    #         arguments.append((morph.surface, str(chuncks[src])))
                    
                    #文節末の助詞だけを記入するなら
                    morph = chuncks[src].morphs[-1]
                    if morph.pos == '助詞':
                        arguments.append((morph.surface, str(chuncks[src])))
                
                if len(arguments) > 0:
                    arguments = sorted(arguments, key=lambda s:s[0])
                    ans.append(verb + '\t' + ' '.join([p[0] for p in arguments]) + '\t' + ' '.join([p[1] for p in arguments]))
                    
                break #最左の動詞だけを取る
                
with open('.\\46.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(ans))

# 47. 機能動詞構文のマイニング

動詞のヲ格にサ変接続名詞が入っている場合のみに着目したい．46のプログラムを以下の仕様を満たすように改変せよ．

    「サ変接続名詞+を（助詞）」で構成される文節が動詞に係る場合のみを対象とする
    述語は「サ変接続名詞+を+動詞の基本形」とし，文節中に複数の動詞があるときは，最左の動詞を用いる
    述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
    述語に係る文節が複数ある場合は，すべての項をスペース区切りで並べる（助詞の並び順と揃えよ）

例えば「また、自らの経験を元に学習を行う強化学習という手法もある。」という文から，以下の出力が得られるはずである．

学習を行う	に を	元に 経験を

In [39]:
ans = []

for chuncks in sentences:
    for i, chunck in enumerate(chuncks):
        morphs = chunck.morphs
        if len(morphs)==2 and morphs[0].pos=='名詞' and morphs[0].pos1=='サ変接続' and morphs[1].base=='を' and morphs[1].pos=='助詞':
            verb = ''
            for morph in chuncks[chunck.dst].morphs:
                if morph.pos=='動詞':
                    verb = morph.base
                    break
            if verb:
                verb_phrase = str(chunck) + verb
                arguments = []
                for src in chuncks[chunck.dst].srcs:
                    if src != i:  
                        
                        #すべての助詞を記入するなら
                        # for morph in chuncks[src].morphs:
                        #     if morph.pos == '助詞':
                        #         arguments.append((morph.surface, str(chuncks[src])))
                        
                        #文節末の助詞だけを記入するなら
                        morph = chuncks[src].morphs[-1]
                        if morph.pos == '助詞':
                            arguments.append((morph.surface, str(chuncks[src])))
                       
                if len(arguments) > 0:
                    arguments = sorted(arguments, key=lambda s:s[0])
                    ans.append(verb_phrase + '\t' + ' '.join([p[0] for p in arguments]) + '\t' + ' '.join([p[1] for p in arguments]))
                    
with open('.\\47.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(ans))

# 48. 名詞から根へのパスの抽出

文中のすべての名詞を含む文節に対し，その文節から構文木の根に至るパスを抽出せよ． ただし，構文木上のパスは以下の仕様を満たすものとする．

    各文節は（表層形の）形態素列で表現する
    パスの開始文節から終了文節に至るまで，各文節の表現を” -> “で連結する

「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． CaboChaを係り受け解析に用いた場合，次のような出力が得られると思われる．

ジョンマッカーシーは -> 作り出した
AIに関する -> 最初の -> 会議で -> 作り出した
最初の -> 会議で -> 作り出した
会議で -> 作り出した
人工知能という -> 用語を -> 作り出した
用語を -> 作り出した

KNPを係り受け解析に用いた場合，次のような出力が得られると思われる．

ジョンマッカーシーは -> 作り出した
ＡＩに -> 関する -> 会議で -> 作り出した
会議で -> 作り出した
人工知能と -> いう -> 用語を -> 作り出した
用語を -> 作り出した

In [45]:
chuncks = sentences[1]
ans = []

for chunck in chuncks:
    if '名詞' in [morph.pos for morph in chunck.morphs]:
        path = [str(chunck)]
        current_chunck = chunck
        while current_chunck.dst != -1:
            current_chunck = chuncks[current_chunck.dst]
            path.append(str(current_chunck))
        ans.append('->'.join(path))

print('\n'.join(ans))

人工知能->語->研究分野とも->される
じんこうちのう->語->研究分野とも->される
AI->エーアイとは->語->研究分野とも->される
エーアイとは->語->研究分野とも->される
計算->という->道具を->用いて->研究する->計算機科学->の->一分野を->指す->語->研究分野とも->される
概念と->道具を->用いて->研究する->計算機科学->の->一分野を->指す->語->研究分野とも->される
コンピュータ->という->道具を->用いて->研究する->計算機科学->の->一分野を->指す->語->研究分野とも->される
道具を->用いて->研究する->計算機科学->の->一分野を->指す->語->研究分野とも->される
知能を->研究する->計算機科学->の->一分野を->指す->語->研究分野とも->される
研究する->計算機科学->の->一分野を->指す->語->研究分野とも->される
計算機科学->の->一分野を->指す->語->研究分野とも->される
一分野を->指す->語->研究分野とも->される
語->研究分野とも->される
言語の->推論->問題解決などの->知的行動を->代わって->行わせる->技術または->研究分野とも->される
理解や->推論->問題解決などの->知的行動を->代わって->行わせる->技術または->研究分野とも->される
推論->問題解決などの->知的行動を->代わって->行わせる->技術または->研究分野とも->される
問題解決などの->知的行動を->代わって->行わせる->技術または->研究分野とも->される
知的行動を->代わって->行わせる->技術または->研究分野とも->される
人間に->代わって->行わせる->技術または->研究分野とも->される
コンピューターに->行わせる->技術または->研究分野とも->される
技術または->研究分野とも->される
計算機->コンピュータによる->情報処理システムの->実現に関する->研究分野とも->される
コンピュータによる->情報処理システムの->実現に関する->研究分野とも->される
知的な->情報処理システムの->実現に関する->研究分野とも->される
情報処理システムの->実現に関する->研究分野とも->される
設計や->実現に関する->研究分野と

# 49. 名詞間の係り受けパスの抽出

文中のすべての名詞句のペアを結ぶ最短係り受けパスを抽出せよ．ただし，名詞句ペアの文節番号がi
とj（i<j）のとき，係り受けパスは以下の仕様を満たすものとする．

    問題48と同様に，パスは開始文節から終了文節に至るまでの各文節の表現（表層形の形態素列）を” -> “で連結して表現する
    文節iとjに含まれる名詞句はそれぞれ，XとYに置換する

また，係り受けパスの形状は，以下の2通りが考えられる．

    文節iから構文木の根に至る経路上に文節jが存在する場合: 文節iから文節jのパスを表示
    上記以外で，文節iと文節jから構文木の根に至る経路上で共通の文節kで交わる場合: 文節iから文節kに至る直前のパスと文節jから文節kに至る直前までのパス，文節kの内容を” | “で連結して表示

In [113]:
# https://amaru-ai.com/entry/2022/10/12/200121を参考にした

from itertools import combinations
import re

ans = []
nouns = []

for i, chunck in enumerate(chuncks):
    #if chunck.contain_pos('名詞'):
    if chunck.is_noun_phrase():
        nouns.append(i)

for i, j in combinations(nouns, 2):
    path_1 = []
    path_2 = []
    while i != j:
        if i < j:
            path_1.append(i)
            i = chuncks[i].dst
        else:
            path_2.append(j)
            j = chuncks[j].dst
    if len(path_2) == 0:
        # node_x = ''.join([morph.surface if morph.pos != '名詞' else 'X' for morph in chuncks[path_1[0]].morphs])
        # node_y = ''.join([morph.surface if morph.pos != '名詞' else 'Y' for morph in chuncks[i].morphs])
        # node_x = re.sub('X+', 'X', node_x)
        # node_y = re.sub('Y+', 'Y', node_y)
        node_x = 'X' + chuncks[path_1[0]].morphs[-1].surface
        node_y = 'Y' + chuncks[i].morphs[-1].surface
        ans.append('->'.join([node_x] + [str(chuncks[n]) for n in path_1[1:]] + [node_y]))
    else:
        # node_x = ''.join([morph.surface if morph.pos != '名詞' else 'X' for morph in chuncks[path_1[0]].morphs])
        # node_y = ''.join([morph.surface if morph.pos != '名詞' else 'Y' for morph in chuncks[path_2[0]].morphs])
        # node_x = re.sub('X+', 'X', node_x)
        # node_y = re.sub('Y+', 'Y', node_y)
        node_x = 'X' + chuncks[path_1[0]].morphs[-1].surface
        node_y = 'Y' + chuncks[path_2[0]].morphs[-1].surface
        node_common = str(chuncks[i])
        path_x_to_common = '->'.join([node_x] + [str(chuncks[n]) for n in path_1[1:]])
        path_y_to_common = '->'.join([node_y] + [str(chuncks[n]) for n in path_2[1:]])
        ans.append('|'.join([path_x_to_common, path_y_to_common, node_common]))